In [1]:
import numpy as np

import pandas as pd
import pandas_ta as ta # pandas technical analysis
import pandas_datareader.data as web

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,6)
plt.style.use("classic") # plt.style.available[:] gives list of all available stylesheets
plt.rcParams["font.size"] = 11
plt.rcParams["lines.linestyle"] = "-"
plt.rcParams["figure.dpi"] = 700 #sharp image

import seaborn as sns

from datetime import datetime

import yfinance as yf

from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import MACD

# from sklearn.tree import DecisionTreeRegressor
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression

# from sklearn.svm import SVC # svm
# from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, accuracy_score
# from sklearn.model_selection import GridSearchCV # c, gamma parameter optimisation

In [2]:
end = datetime.today()
start = datetime(end.year-1, end.month, end.day)

ticker = ["^GSPC", 
          "^IXIC"
         ] # add tickers here

df = web.DataReader(ticker, "yahoo", start, end)

In [3]:
# .stack() converts the innermost column index into the innermost row index
df = df.stack() # https://medium.com/swlh/reshaping-in-pandas-with-stack-and-unstack-functions-bb169f64467d
df = df.reset_index() # not necessary, puts the same date to each innermost row index
df

Attributes,Date,Symbols,Adj Close,Close,High,Low,Open,Volume
0,2020-09-14,^GSPC,3383.540039,3383.540039,3402.929932,3363.560059,3363.560059,3832130000
1,2020-09-14,^IXIC,11056.650391,11056.650391,11118.290039,10982.259766,11010.139648,3716820000
2,2020-09-15,^GSPC,3401.199951,3401.199951,3419.479980,3389.250000,3407.729980,4051460000
3,2020-09-15,^IXIC,11190.320312,11190.320312,11244.459961,11127.980469,11193.959961,3772820000
4,2020-09-16,^GSPC,3385.489990,3385.489990,3428.919922,3384.449951,3411.229980,4710030000
...,...,...,...,...,...,...,...,...
499,2021-09-09,^IXIC,15248.250000,15248.250000,15352.379883,15245.169922,15296.059570,3997250000
500,2021-09-10,^GSPC,4458.580078,4458.580078,4520.470215,4457.660156,4506.919922,2851140000
501,2021-09-10,^IXIC,15115.490234,15115.490234,15349.469727,15111.309570,15332.919922,4567980000
502,2021-09-13,^GSPC,4468.729980,4468.729980,4492.990234,4445.700195,4474.810059,3096390000


In [6]:
class TechnicalIndicators:
    """
    This is an example of the TechIndicator class.
    """
    def __init__(self, symbol, price):
        self.symbol = symbol
        self.price = price
        self.technical_indicators = None
        
    def set_technical_indicators(self):
        self.technical_indicators = pd.DataFrame()
        self.technical_indicators["Adj Close"] = self.price
    
    def macd(self):
        """
        The moving average convergence/divergence.
        """
        dataframe = self.technical_indicators
        closing_price = self.price
        ema_26 = 26
        ema_12 = 12
        signal = 9
        macd = MACD(closing_price, 
                    window_fast = ema_12, 
                    window_slow = ema_26,
                    window_sign = signal
                   )
        dataframe["MACD"] = macd.macd()
        
        return dataframe
    
    def rsi(self):
        """
        The relative strength index.
        """
        closing_price = self.price
        dataframe = self.technical_indicators
        rsi_time_period = 14
        
        rsi = RSIIndicator(close = closing_price, 
                           window = rsi_time_period
                          )
        
        dataframe["RSI"] = rsi.rsi()
        
        return dataframe

    def bollinger_bands(self):
        """
        Bollinger bands.
        """
        closing_price = self.price
        dataframe = self.technical_indicators
        window = 20
        
        bbands = BollingerBands(close = closing_price,
                               window = window,
                               window_dev = 2
                               )
        
        dataframe["Bollinger Bands Middle"] = bbands.bollinger_mavg()
        dataframe["Bollinger Bands Lower"] = bbands.bollinger_hband()
        dataframe["Bollinger Bands Upper"] = bbands.bollinger_lband()
        
        return dataframe
    
    def moving_average(self):
        """
        Moving Average.
        """
        dataframe = self.technical_indicators
        days = [5,10,20]
        
        for i in days:
            col_name = f"MA for {i} days"
            dataframe[col_name] = self.price.rolling(window = i).mean()
            
        return dataframe

    def rate_change(self, period):
        """
        Rate of Change.
        """
        dataframe = self.technical_indicators
        dataframe["Rate of Change"] = ((self.price - self.price.shift(period))/self.price.shift(period)) * 100
        
        return dataframe

In [7]:
gspc = df[df["Symbols"] == "^GSPC"]
ixic = df[df["Symbols"] == "^IXIC"]

symbol = TechnicalIndicators(gspc, gspc["Adj Close"])
symbol.set_technical_indicators()
symbol.rsi()
symbol.bollinger_bands()
symbol.rate_change(30).tail(40)

,Adj Close,RSI,Bollinger Bands Middle,Bollinger Bands Lower,Bollinger Bands Upper,Rate of Change
424,4258.490234,43.718267,4313.915454,4410.777836,4217.053072,0.676143
426,4323.060059,53.556710,4318.828955,4406.657241,4231.000669,2.284150
428,4358.689941,57.927197,4324.441455,4407.251799,4241.631111,3.109110
430,4367.479980,58.953427,4330.723462,4406.258595,4255.188329,3.505828
432,4411.790039,63.753089,4337.988452,4415.342266,4260.634639,4.071775
434,4422.299805,64.804321,4345.068433,4425.990617,4264.146248,4.116830
436,4401.459961,61.024938,4350.610937,4431.037153,4270.184722,3.438423
438,4400.640137,60.874532,4356.052954,4434.530647,4277.575261,3.627624
440,4419.149902,63.086632,4362.135449,4440.374293,4283.896606,4.627452
442,4395.259766,58.490103,4365.901440,4442.894542,4288.908338,4.107192


In [ ]:
help(ta.rsi)

In [ ]:
def p_twinx():
    """
    The below code plots two lineplots on the same chart. 
    """
    fig, ax = plt.subplots(1,1,figsize=(15,4), constrained_layout=True)
    ax.plot(gspc["Date"], gspc["Adj Close"], color="blue")
    ax.set(title="^GSPC/^IXIC")

    ax2 = ax.twinx()
    ax2.plot(ixic["Date"], ixic["Adj Close"], color="red")

    ax.tick_params("y", colors="blue")
    ax2.tick_params("y", colors="red")

In [ ]:
def p():
    fig, (ax1, ax2, ax3) = plt.subplots(3,1,figsize=(15,10), constrained_layout=True)
    ax1.plot(gspc["Date"], gspc["Adj Close"], color="blue")
    ax1.set(title="^GSPC")
    
    ax2.plot(ixic["Date"], ixic["Adj Close"])
    ax2.set(title="^IXIC")
    
    ax3.plot(n225["Date"], n225["Adj Close"])
    ax3.set(title="^N225")

    fig.tight_layout(h_pad=2)

In [ ]:
# used for local change only, rcParams used for global change
with plt.style.context("seaborn-ticks"):
    p_twinx()